# HTTP Requests

In this part I will be taking all the unique gids of each recording to make a request to the AcousticBrainz server to obtain their json files. This process has been parallelized to download all the data in chunks so if there is a failure in one of the chunks I can redownload that specific chunk of data. 

I can make requests for 25 recordings at a time which is why I'm clustering the gids into clusters of 25 using the function **knitclust()**. This will produce a gid string consisting of 25 gids separated by ";", although the requests interprets the semi-colon as a "%" it works just fine.

I have split the data up into array of 48 which will give me 48 chunks. In those arrays, I have further broken the gid clusters into chunks based on the number of processes to speed things up. Ultimately, I was limited by the server for reaching maximum requests.

once the json is obtained I then simply export it to a json file in a folder for further processing. In the next part I will collapse this json file into either a csv or parquet. 

The json file is in the format {gid1: json_data, gid2: json_data2, ...}.

also note that MusicBrainz has more gids than AcousticBrainz has data for them. The amount of data you can expect to aquire is around 23%.

### Slurm Job

In [ ]:
#!/bin/bash
#SBATCH --time=1:00:00
#SBATCH -N 2
#SBATCH -n 48
#SBATCH -a 0-47

module load python/anaconda-3.6-5.1.0
python areq.py $SLURM_ARRAY_TASK_ID $SLURM_ARRAY_TASK_COUNT

#### contents of areq.py

In [ ]:
import sys, os, time
import numpy as np
import pandas as pd
import math
import json
import requests
from requests.auth import HTTPBasicAuth
from multiprocessing import Pool, Process, Queue, cpu_count

DATAPATH = 'reportfiles/main_genre.parquet'

#provide a username and password to increase the rate limit.
#the rate limit is the bottle neck so providing a username and password for AcousticBrainz will increase the speed
USERNAME = 'put your username here'
PASSWORD = 'put your password here'

def knitclust(data):
	splitnum = math.ceil(data.shape[0]/25)
	clust = np.array_split(list(data['gid']), splitnum) # [[25],[25],...]
	cleanurl = []
	for row in clust:
		gids = ''
		for gid in row:
			gids += gid + ';'
		cleanurl.append(gids[:-1])
	return cleanurl

def get_data(urlclust, out_q):
	url = "https://acousticbrainz.org/api/v1/low-level"
	datdict = {}
	for ids in urlclust:
		mbid = {"recording_ids": ids}
		r = requests.get(url=url, params = mbid, auth=HTTPBasicAuth(USERNAME, PASSWORD))

        #If you get a rate limit error I'll stall for the required seconds plus 10 just in case
        #it resets every 10 seconds but I have added a buffer
		if r.status_code >= 400:
			print(r.status_code)
			sec = int(r.headers['X-RateLimit-Reset-In'])
			print("*** STALLED FOR {} SECONDS ***".format(sec))
			time.sleep(sec+10)
			r = requests.get(url=url, params = mbid, auth=HTTPBasicAuth(USERNAME, PASSWORD))
			pass

        #The error handling here isn't be best but if there is a problem with a chunk of data I'll know
		r.raise_for_status()

		datdict.update(r.json())
	out_q.put(datdict)

if __name__=="__main__":
	taskid = int(sys.argv[1])
	ntasks = int(sys.argv[2])
	nproc = int(os.environ["SLURM_CPUS_ON_NODE"])

	testdata = pd.read_parquet(DATAPATH)
	print("read success")
	procs = []
	findict = {}
	out_q = Queue()

	taskchunks = np.array_split(knitclust(testdata), ntasks) #split gid clusters into the number of tasks
	newfiles = np.array_split(taskchunks[taskid], nproc) # split it further using processes to speed things up

	for i in range(nproc):
		p = Process(target=get_data, args = (newfiles[i], out_q))
		procs.append(p)
		p.start()
	for j in range(nproc):
		findict.update(out_q.get()) #update the dictionary
	for p in procs:
		p.join()

	with open('data/chunk_{}.json'.format(taskid), 'w') as fp:
		json.dump(findict, fp)
	
	print("write finished")

## What does this look like?
let's make a chunk of gids

In [4]:
import pandas as pd
import numpy as np
import math

def knitclust(data):
	splitnum = math.ceil(data.shape[0]/25)
	clust = np.array_split(list(data['gid']), splitnum) # [[25],[25],...]
	cleanurl = []
	for row in clust:
		gids = ''
		for gid in row:
			gids += gid + ';'
		cleanurl.append(gids[:-1])
	return cleanurl

mdf = pd.read_parquet('reportfiles/main_genre.parquet')
knitclust(mdf)[0]

'00000baf-9215-483a-8900-93756eaf1cfc;000026d2-8db1-42b1-87da-e4389dcd6093;00007908-1fff-415d-8e87-a49722c2442b;00007960-9d81-4192-b548-ad33d6b0ca54;00007bab-7268-41c4-9d5c-c335c3a26f7c;000080ea-f4d1-41c6-a327-76280f90d39f;00008c47-0ba9-4706-b757-ce34c866e41d;0000b40f-8dac-4d28-804a-3d2a52c6e8b8;0000db7e-dbd0-4e54-a2d5-230a0b708c19;0000eb3d-2690-4f3e-b3ac-6c8683c0d765;0000f9f6-4070-4dd0-a7f0-591d64e236ef;0001004a-b5fb-4923-9d9c-f83c53e3d14d;0001093f-3ca1-42c8-bcc7-8fbbcd2f7c72;00012695-cefb-4dbf-a1a3-64238193a2b8;000135b5-5077-4516-bbed-6448de5a17b8;00014fc3-e892-4437-beb0-c3eb81091239;000160ee-1bc8-4a98-89a4-1aa4789a09a7;0001b138-11eb-4b9f-9f5c-c0dfe2b1f87b;0001cd44-0521-4989-b73c-c8931653a217;0001d90b-e0f7-4e6a-bb17-aa176f0dba00;0001fc6f-e494-4c88-86e0-ff31c7fcf173;00021513-1c17-49e9-95c3-693a86b7ed1a;000252d8-b80e-41d4-80a1-d8a9846ca27a;0002b951-4fd1-4a99-bb20-6c8fc648d73b;0002bdf8-6ade-481d-b6c8-eb01ad89206d'

That string of gids will be shoved into the url https://acousticbrainz.org/api/v1/low-level to obtain the data like so:  
https://acousticbrainz.org/api/v1/low-level?recording_ids=00000baf-9215-483a-8900-93756eaf1cfc;000026d2-8db1-42b1-87da-e4389dcd6093

that's what get data does.